In [1]:
import sys; sys.path.insert(0, '..')
import time
from tqdm import tqdm_notebook as tqdm

v = '20180429'

In [2]:
from PyQt5.QtWidgets import QApplication
import kiwoom

app = QApplication(sys.argv)
hts = kiwoom.Kiwoom()

[2018-05-01 08:33:22,507][INFO] 키움증권 API 객체 준비 완료


In [3]:
for _ in range(5):
    if not hts.kiwoom_GetConnectState():
        hts.kiwoom_CommConnect()
        time.sleep(5)
        print('타임아웃 재시도')
    else:
        print('이미 로그인된 상태')
        break

타임아웃 재시도


[2018-05-01 08:33:29,370][INFO] 로그인 성공


타임아웃 재시도
이미 로그인된 상태


In [4]:
import pandas as pd
df = pd.read_csv('../data/all_{v}.csv'.format(v=v), header=0, converters={'종목코드': lambda x: str(x)})

In [9]:
dict_datalist = {}

def add_data(data):
    for k, v in data.items():
        if k not in dict_datalist:
            dict_datalist[k] = []
        dict_datalist[k].append(v)
        
hts.dict_callback['주식기본정보'] = add_data
for i in tqdm(range(len(df))):
    row = df.iloc[i]
    code = row['종목코드']
    hts.kiwoom_TR_OPT10001_주식기본정보요청(code)
    time.sleep(2)

A Jupyter Widget

In [10]:
df2 = pd.DataFrame(data=dict_datalist)
df2.to_csv('../data/all_basic_{v}.csv'.format(v=v), index=False, encoding='utf-8')

In [21]:
list_ranking_col = ['PBR', 'PER']
list_ranking_asc = [True, True]
list_rank = list(range(1, len(df2)+1))
df3 = df2
for col, asc in zip(list_ranking_col, list_ranking_asc):
    df3 = df3.sort_values(by=col, ascending=asc)
    df3['rank_'+col] = list_rank
df3 = df3.sort_index()

In [22]:
df3['rank_PBR_PER'] = (df3['rank_PBR'] + df3['rank_PER']) / 2

In [37]:
df4['PBR'].convert_objects(convert_numeric=True)

C:\Users\mkk\Anaconda3x86\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


181     0.00
695     0.00
419     0.00
180     0.00
269     0.00
176     0.00
442     0.00
745     0.00
232     0.00
168     0.00
644     0.44
228     0.00
300     0.00
581     0.00
414     0.44
326     0.00
592     0.00
294     0.00
127     0.00
480     0.00
514     0.00
382     0.00
612     0.00
364     0.22
629     0.00
445     0.00
370     0.00
13      0.00
635     0.00
638     0.00
       ...  
668     1.69
347     3.04
461     2.13
334     1.42
450     2.17
472     3.02
19      2.28
630     1.58
45      1.54
481     3.31
62      3.41
657     1.83
483     7.48
692    15.29
411     9.59
225     5.91
623     4.21
165     1.62
367     2.00
218     9.62
624     4.13
283     3.34
765     7.64
251     3.46
727     5.90
38      2.73
738     2.90
353     5.97
591     6.24
458     6.54
Name: PBR, Length: 777, dtype: float64

In [36]:
df4 = df3.sort_values(by='rank_PBR_PER')
df4 = df4[df4['PBR'] >= 0.2]
df4['종목명']

TypeError: '>=' not supported between instances of 'str' and 'float'